# Training Model

## Setup MLFlow

In [ ]:
# # SETUP BLOCK (Run once per project lifetime)
# import mlflow
# import os

# # 1. Define Absolute Paths
# project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
# db_uri = f"sqlite:///{os.path.join(project_root, 'mlflow.db')}"
# artifact_uri = "file:///" + os.path.join(project_root, "mlruns").replace("\\", "/")

# # 2. Connect
# mlflow.set_tracking_uri(db_uri)

# # 3. Create (Safe Check)
# if not mlflow.get_experiment_by_name("CreditGuard"):
#     mlflow.create_experiment("CreditGuard", artifact_location=artifact_uri)

2026/02/18 21:35:58 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/18 21:35:58 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/18 21:35:58 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/18 21:35:58 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/18 21:35:58 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/18 21:35:58 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/18 21:35:59 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/18 21:35:59 INFO mlflow.store.db.utils: Updating database tables
2026/02/18 21:35:59 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/18 21:35:59 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/18 21:35:59 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/02/18 21:3

In [1]:
import mlflow

mlflow.set_tracking_uri("sqlite:///../mlflow.db")

mlflow.set_experiment("CreditGuard")

2026/02/22 20:21:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/22 20:21:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/22 20:21:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/22 20:21:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/22 20:21:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/22 20:21:35 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/22 20:21:35 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/22 20:21:35 INFO alembic.runtime.migration: Will assume non-transactional DDL.


<Experiment: artifact_location='file:///d:/projects/CreditGuard/mlruns', creation_time=1771421761464, experiment_id='1', last_update_time=1771421761464, lifecycle_stage='active', name='CreditGuard', tags={'mlflow.experimentKind': 'custom_model_development'}>

## EXP#1

- using only the cleaned dataset
  - imputed the missing value with mode (others with NA below ten is dropped)
- did FE only for encoding and converting to datetime
- classes is VERY imbalanced
- using random forest bcs
  - It handles "Messy" Math
  - It's Robust
  - It tells you "Why"


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report
)

FE_DATA_PATH = '../data/processed/lending-club-fe.csv'
TARGET_COL = 'loan_status'

In [5]:
df = pd.read_csv(FE_DATA_PATH)
print(df.shape)

(2436, 36)


In [14]:
# Check the Total Balance
print("--- Total Target Balance ---")
print(df['loan_status'].value_counts())
print(df['loan_status'].value_counts(normalize=True))

# Check the size again
print(f"\nTotal Rows: {len(df)}")

--- Total Target Balance ---
loan_status
Fully Paid     2417
Charged Off      19
Name: count, dtype: int64
loan_status
Fully Paid     0.9922
Charged Off    0.0078
Name: proportion, dtype: float64

Total Rows: 2436


In [15]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,issue_d,loan_status,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies,credit_history_years,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,initial_list_status_w,application_type_Joint App
0,30000,0,22.35,1151.16,3,19,5,100000.0,2,2,Fully Paid,30.46,11,1,15603,37.0,19,1,1,3.745380,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True
1,40000,1,16.14,975.71,2,13,0,45000.0,1,2,Fully Paid,50.53,18,0,34971,64.5,37,1,0,6.329911,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True
2,20000,0,7.56,622.68,0,2,10,100000.0,0,2,Fully Paid,18.92,9,0,25416,29.9,19,5,0,16.659822,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True
3,4500,0,11.31,147.99,1,7,10,38500.0,0,2,Fully Paid,4.64,12,0,4472,15.3,25,0,0,11.830253,False,False,True,True,False,False,False,False,False,False,False,False,False,False,True,False
4,8425,0,27.27,345.18,4,24,3,450000.0,1,2,Fully Paid,12.37,21,0,36812,65.7,37,4,0,17.995893,True,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2436 entries, 0 to 2435
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   2436 non-null   int64  
 1   term                        2436 non-null   int64  
 2   int_rate                    2436 non-null   float64
 3   installment                 2436 non-null   float64
 4   grade                       2436 non-null   int64  
 5   sub_grade                   2436 non-null   int64  
 6   emp_length                  2436 non-null   int64  
 7   annual_inc                  2436 non-null   float64
 8   verification_status         2436 non-null   int64  
 9   issue_d                     2436 non-null   int64  
 10  loan_status                 2436 non-null   object 
 11  dti                         2436 non-null   float64
 12  open_acc                    2436 non-null   int64  
 13  pub_rec                     2436 

### Split the dataset

In [6]:
df['loan_status'] = df['loan_status'].map({'Fully Paid': 0, 'Charged Off': 1})

In [7]:
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

In [8]:
# Split off the TEST set (10% of total)
# stratify=y ensures the 80/20 ratio is preserved in the test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42, stratify=y
)

# Split the remaining 90% into TRAIN (80%) and VALIDATION (10%)
# The math: 0.10 / 0.90 = 0.1111...
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42, stratify=y_train_val
)

print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f"Val shapes:   {X_val.shape}, {y_val.shape}")
print(f"Test shapes:  {X_test.shape}, {y_test.shape}")

Train shapes: (1948, 35), (1948,)
Val shapes:   (244, 35), (244,)
Test shapes:  (244, 35), (244,)


### MLFlow Params Definition

In [9]:
# Model Hyperparameters
model_params = {
    "n_estimators": 100,
    "max_depth": 10,
    "class_weight": "balanced",
    "random_state": 42
}

# Data Processing
# Happened in Notebook 01 & 02
data_params = {
    "cleaning_missing_strategy": "drop_rows | mode_impute",
    "cleaning_outlier_removal": "manual_cap_dti_100",
    "fe_encoding": "ordinal | one_hot",     
    "fe_balance_strategy": "none",
    "data_split_ratio": "80/10/10"
}

### Training

In [10]:
with mlflow.start_run(run_name="EXP_01_Baseline_Logic"):
    
    # --- Log Hyperparameters & Data Preprocessing ---
    mlflow.log_params(model_params)
    mlflow.log_params(data_params)
    
    # --- Log Train Data ---
    # Log the shape so to see added/removed columns
    mlflow.log_param("input_rows", X_train.shape[0])
    mlflow.log_param("input_cols", X_train.shape[1])
    mlflow.log_param("column_names", X_train.columns.tolist())

    # --- Log Notebooks --- 
    # This uploads the actual cleaning code to MLflow.
    # If Notebook is overrided, we still have THIS version saved here.
    mlflow.log_artifact("01_eda_and_cleaning.ipynb", artifact_path="code_snapshot")
    mlflow.log_artifact("02_feature_engineering.ipynb", artifact_path="code_snapshot")

    # --- Train & Evaluate ---
    print("Training Model...")
    rf_model = RandomForestClassifier(**model_params)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    y_prob = rf_model.predict_proba(X_test)[:, 1] # Probability of Class 1 (Charged Off)

    # --- Log Metrics ---
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1_score": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_prob)
    }
    mlflow.log_metrics(metrics)
    print(f"Logged Metrics: {metrics}")
    
    # --- Log Model ---
    mlflow.sklearn.log_model(rf_model, artifact_path="model")
    
    # --- Log Confusion Matrix ---
    cm = confusion_matrix(y_test, y_pred)    
    fig = plt.figure(figsize=(6,5))
    
    labels = ['Fully Paid', 'Charged Off']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix - EXP_01')
    
    mlflow.log_figure(fig, "confusion_matrix.png")
    
    plt.close(fig)
    
    print("Run Complete. Notebooks and Params saved to MLflow.")

2026/02/18 21:36:51 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Training Model...
Logged Metrics: {'accuracy': 0.9918032786885246, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0, 'roc_auc': 0.5475206611570248}


c:\Users\USER\miniconda3\envs\python10\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Run Complete. Notebooks and Params saved to MLflow.


## EXP#2

- handled missing value
  - emp_length's na is replaced w/ 'Unknown' -> after FE imputed with mode + flagging
  - mort_acc's na is imputed with median + flagging
  - others with NA below ten is dropped
- improved FE on encoding (added target encoding, made few adjustments) and converting to datetime (+issue_d)
- classes is less imbalanced than EXP#1 (99/1 -> 80/20)
- using random forest bcs
  - It handles "Messy" Math
  - It's Robust
  - It tells you "Why"


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report
)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

FE_DATA_PATH = '../data/processed/lending-club-fe.csv'
TARGET_COL = 'loan_status'

In [3]:
df = pd.read_csv(FE_DATA_PATH)
print(df.shape)

(4994, 28)


In [4]:
# Check the Total Balance
print("--- Total Target Balance ---")
print(df['loan_status'].value_counts())
print(df['loan_status'].value_counts(normalize=True))

# Check the size again
print(f"\nTotal Rows: {len(df)}")

--- Total Target Balance ---
loan_status
0    3996
1     998
Name: count, dtype: int64
loan_status
0    0.80016
1    0.19984
Name: proportion, dtype: float64

Total Rows: 4994


In [5]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,verification_status,loan_status,...,pub_rec_bankruptcies,mort_acc_missing,credit_history_years,issue_years,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,initial_list_status_w,application_type_Joint App,emp_length_unknown
0,5000,0,11.53,164.96,1,9,10.0,21000.0,1,0,...,1.0,0,40.668036,3.501711,False,True,False,True,False,1
1,30375,0,13.49,1030.64,2,11,10.0,108000.0,1,0,...,0.0,0,11.668720,2.499658,False,True,False,True,False,0
2,13250,0,18.75,484.02,3,17,10.0,50000.0,2,0,...,0.0,0,26.748802,5.582478,True,False,False,False,False,0
3,7000,0,7.07,216.37,0,1,10.0,50000.0,0,0,...,0.0,0,13.330595,1.415469,True,False,False,True,False,0
4,5000,0,7.29,155.05,0,3,10.0,60000.0,0,0,...,0.0,1,19.997262,7.997262,False,True,False,False,False,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4994 entries, 0 to 4993
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   4994 non-null   int64  
 1   term                        4994 non-null   int64  
 2   int_rate                    4994 non-null   float64
 3   installment                 4994 non-null   float64
 4   grade                       4994 non-null   int64  
 5   sub_grade                   4994 non-null   int64  
 6   emp_length                  4994 non-null   float64
 7   annual_inc                  4994 non-null   float64
 8   verification_status         4994 non-null   int64  
 9   loan_status                 4994 non-null   int64  
 10  purpose                     4994 non-null   float64
 11  dti                         4994 non-null   float64
 12  open_acc                    4994 non-null   float64
 13  pub_rec                     4994 

### Split the dataset

In [7]:
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

In [8]:
# Split off the TEST set (10% of total)
# stratify=y ensures the 80/20 ratio is preserved in the test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42, stratify=y
)

# Split the remaining 90% into TRAIN (80%) and VALIDATION (10%)
# The math: 0.10 / 0.90 = 0.1111...
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42, stratify=y_train_val
)

print(f"Train shapes: {X_train.shape}, {y_train.shape}")
print(f"Val shapes:   {X_val.shape}, {y_val.shape}")
print(f"Test shapes:  {X_test.shape}, {y_test.shape}")

Train shapes: (3994, 27), (3994,)
Val shapes:   (500, 27), (500,)
Test shapes:  (500, 27), (500,)


### MLFlow Params Definition

In [9]:
# Model Hyperparameters
rf_params = {
    "n_estimators": 100,
    "max_depth": 10,
    "min_samples_split": 10,
    "min_samples_leaf": 4,
    "class_weight": "balanced",
    "random_state": 42,
    "n_jobs": -1
}
xgb_params = {
    "n_estimators": 100,
    "max_depth": 6,
    "learning_rate": 0.1,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 4,
    "random_state": 42,
    "n_jobs": -1,
    "eval_metric": "logloss"
}
lgbm_params = {
    "n_estimators": 100,
    "max_depth": 7,
    "learning_rate": 0.1,
    "num_leaves": 31,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "scale_pos_weight": 4,
    "random_state": 42,
    "n_jobs": -1,
    "verbose": -1
}

# Data Processing
# Happened in Notebook 01 & 02
data_params = {
    "cleaning_missing_strategy": "drop_rows | unknown_fill > mode_impute | median_impute",
    "cleaning_outlier_removal": "none",
    "fe_encoding": "ordinal | one_hot | target",     
    "fe_balance_strategy": "none",
    "data_split_ratio": "80/10/10"
}

### Training

Random Forest

In [10]:
with mlflow.start_run(run_name="EXP_02a_RF_MoreData_PreprocV2"):
    
    # --- Log Hyperparameters & Data Preprocessing ---
    mlflow.log_params(rf_params)
    mlflow.log_params(data_params)
    
    # --- Log Train Data ---
    # Log the shape so to see added/removed columns
    mlflow.log_param("input_rows", X_train.shape[0])
    mlflow.log_param("input_cols", X_train.shape[1])
    mlflow.log_param("column_names", X_train.columns.tolist())

    # --- Log Notebooks --- 
    # This uploads the actual cleaning code to MLflow.
    # If Notebook is overrided, we still have THIS version saved here.
    mlflow.log_artifact("01_eda_and_cleaning.ipynb", artifact_path="code_snapshot")
    mlflow.log_artifact("02_feature_engineering.ipynb", artifact_path="code_snapshot")

    # --- Train & Evaluate ---
    print("Training Model...")
    rf_model = RandomForestClassifier(**rf_params)
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)
    y_prob = rf_model.predict_proba(X_test)[:, 1] # Probability of Class 1 (Charged Off)

    # --- Log Metrics ---
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1_score": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_prob)
    }
    mlflow.log_metrics(metrics)
    print(f"Logged Metrics: {metrics}")
    
    # --- Log Model ---
    mlflow.sklearn.log_model(rf_model, artifact_path="model")
    
    # --- Log Confusion Matrix ---
    cm = confusion_matrix(y_test, y_pred)    
    fig = plt.figure(figsize=(6,5))
    
    labels = ['Fully Paid', 'Charged Off']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix - EXP_02a (RF)')
    
    mlflow.log_figure(fig, "confusion_matrix.png")
    
    plt.close(fig)
    
    print("Run Complete. Notebooks and Params saved to MLflow.")

Training Model...


2026/02/22 20:27:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logged Metrics: {'accuracy': 0.722, 'precision': 0.3211009174311927, 'recall': 0.35, 'f1_score': 0.3349282296650718, 'roc_auc': 0.6405}


c:\Users\USER\miniconda3\envs\python10\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Run Complete. Notebooks and Params saved to MLflow.


In [11]:
with mlflow.start_run(run_name="EXP_02b_XGB_MoreData_PreprocV2"):
    
    # --- Log Hyperparameters & Data Preprocessing ---
    mlflow.log_params(xgb_params)
    mlflow.log_params(data_params)
    
    # --- Log Train Data ---
    # Log the shape so to see added/removed columns
    mlflow.log_param("input_rows", X_train.shape[0])
    mlflow.log_param("input_cols", X_train.shape[1])
    mlflow.log_param("column_names", X_train.columns.tolist())

    # --- Log Notebooks --- 
    # This uploads the actual cleaning code to MLflow.
    # If Notebook is overrided, we still have THIS version saved here.
    mlflow.log_artifact("01_eda_and_cleaning.ipynb", artifact_path="code_snapshot")
    mlflow.log_artifact("02_feature_engineering.ipynb", artifact_path="code_snapshot")

    # --- Train & Evaluate ---
    print("Training Model...")
    xgb_model = XGBClassifier(**xgb_params)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    y_prob = xgb_model.predict_proba(X_test)[:, 1] # Probability of Class 1 (Charged Off)

    # --- Log Metrics ---
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1_score": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_prob)
    }
    mlflow.log_metrics(metrics)
    print(f"Logged Metrics: {metrics}")
    
    # --- Log Model ---
    mlflow.sklearn.log_model(xgb_model, artifact_path="model")
    
    # --- Log Confusion Matrix ---
    cm = confusion_matrix(y_test, y_pred)    
    fig = plt.figure(figsize=(6,5))
    
    labels = ['Fully Paid', 'Charged Off']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix - EXP_02b (XGB)')
    
    mlflow.log_figure(fig, "confusion_matrix.png")
    
    plt.close(fig)
    
    print("Run Complete. Notebooks and Params saved to MLflow.")

Training Model...


2026/02/22 20:27:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logged Metrics: {'accuracy': 0.7, 'precision': 0.30158730158730157, 'recall': 0.38, 'f1_score': 0.336283185840708, 'roc_auc': 0.60095}


c:\Users\USER\miniconda3\envs\python10\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Run Complete. Notebooks and Params saved to MLflow.


In [13]:
with mlflow.start_run(run_name="EXP_02c_LGBM_MoreData_PreprocV2"):
    
    # --- Log Hyperparameters & Data Preprocessing ---
    mlflow.log_params(lgbm_params)
    mlflow.log_params(data_params)
    
    # --- Log Train Data ---
    # Log the shape so to see added/removed columns
    mlflow.log_param("input_rows", X_train.shape[0])
    mlflow.log_param("input_cols", X_train.shape[1])
    mlflow.log_param("column_names", X_train.columns.tolist())

    # --- Log Notebooks --- 
    # This uploads the actual cleaning code to MLflow.
    # If Notebook is overrided, we still have THIS version saved here.
    mlflow.log_artifact("01_eda_and_cleaning.ipynb", artifact_path="code_snapshot")
    mlflow.log_artifact("02_feature_engineering.ipynb", artifact_path="code_snapshot")

    # --- Train & Evaluate ---
    print("Training Model...")
    lgbm_model = LGBMClassifier(**lgbm_params)
    lgbm_model.fit(X_train, y_train)
    y_pred = lgbm_model.predict(X_test)
    y_prob = lgbm_model.predict_proba(X_test)[:, 1] # Probability of Class 1 (Charged Off)

    # --- Log Metrics ---
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, zero_division=0),
        "recall": recall_score(y_test, y_pred, zero_division=0),
        "f1_score": f1_score(y_test, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_test, y_prob)
    }
    mlflow.log_metrics(metrics)
    print(f"Logged Metrics: {metrics}")
    
    # --- Log Model ---
    mlflow.sklearn.log_model(lgbm_model, artifact_path="model")
    
    # --- Log Confusion Matrix ---
    cm = confusion_matrix(y_test, y_pred)    
    fig = plt.figure(figsize=(6,5))
    
    labels = ['Fully Paid', 'Charged Off']
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix - EXP_02c (LBGM)')
    
    mlflow.log_figure(fig, "confusion_matrix.png")
    
    plt.close(fig)
    
    print("Run Complete. Notebooks and Params saved to MLflow.")

2026/02/22 20:28:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Training Model...
Logged Metrics: {'accuracy': 0.684, 'precision': 0.2803030303030303, 'recall': 0.37, 'f1_score': 0.31896551724137934, 'roc_auc': 0.592375}


c:\Users\USER\miniconda3\envs\python10\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)


Run Complete. Notebooks and Params saved to MLflow.
